### Import Package

Jika belum terdapat library bs4 dan requests, lakukanlah langkah instalasi dibawah ini:

In [ ]:
# Install Library
%pip install bs4
%pip install requests
%pip install selenium
%pip install  openpyxl
%pip install xlsxwriter


In [ ]:
%pip install undetected-chromedriver

In [1]:
from bs4 import BeautifulSoup #digunakan untuk web scraping
import requests #digunakan untuk mendapatkan/mengakses/mendownload Webpage
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import undetected_chromedriver as uc


In [5]:
options = uc.ChromeOptions()

# setting profile
options.user_data_dir = "c:\\temp\\profile"
options.page_load_strategy = 'normal'
options.add_argument('--headless')
# use specific (older) version
driver = uc.Chrome(
    options = options , version_main = 113
    )  # version_main allows to specify your chrome version instead of following chrome global version

#masukkan url yang mau di scrape

kategori = "Perlengkapan Rumah (Utama)"
url_kategori = "https://shopee.co.id/Perlengkapan-Rumah-cat.11043778"
max_page = 9 #max page - 1, each 60 product per page
class_all_product = 'col-xs-2-4 shopee-search-item-result__item' 
class_product_name = '_1yN94N WoKSjC _2KkMCe'
class_price = 'du3pq0'
class_total_sales = 'x+3B8m wOebCz'
class_lokasi = 'mrz-bA'

product_df = pd.DataFrame()
product_list = []

for page in range (0, max_page):    
    print("Kategori : {} page : {}".format(kategori, page))
    url = "{}?page={}&sortBy=sales".format(url_kategori, page)
    driver.get(url)    
    # this is just to ensure that the page is loaded
    # Mengulang scroll ke bawah sampai tidak ada lagi item baru

    elem = driver.find_element(by=By.TAG_NAME, value='body')   
    no_of_pagedowns = 10
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        no_of_pagedowns-=1  

    html = driver.page_source

    # Now, we could simply apply bs4 to html variable
    soup = BeautifulSoup(html, "html.parser")
    #print(soup)
    products = soup.find_all('div', {'class' : class_all_product})
    #print(products)
    qty = 0
    for product in products:
        #product url
        a_tag = product.find('a')
        if a_tag is None or 'href' not in a_tag.attrs:
            url_product = None
        else:            
            url_product  = "https://shopee.co.id" + a_tag['href']
        
        #product name
        namaproduct_div = product.find('div', {'class': class_product_name})
        if namaproduct_div is not None:
            namaproduct = namaproduct_div.text
        else:
            namaproduct = None           
        
        #product price
        hargaproduct = product.find_all('span', {'class' :class_price })
        ctr = 0   
        harga1 = 0
        harga2 = 0

        for harga in hargaproduct:            
            if ctr == 0 :
                harga1 = harga.text
            else :
                harga2 = harga.text
                
            ctr += 1

        #total sales
        sales = product.find('div', {'class' : class_total_sales})
        if(sales is None):
            terjualperbulan = 0
        else:   
            terjualperbulan = sales.text

        #lokasi seller
        lokasi = product.find('div', {'class' : class_lokasi}).text

        dict = {"url":url_product, "kategori":kategori,"namaproduct":namaproduct, "hargaproduct1":harga1,"hargaproduct2":harga2,"terjualperbulan":terjualperbulan,"lokasi":lokasi}            
        product_list.append(dict)      
        qty += 1


product_list_df = pd.DataFrame(product_list)
driver.close() # closing the webdriver
product_list_df.to_csv("datashopee_kategori_utama_{}.csv".format(kategori))

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\undetected_chromedriver\__init__.py:326: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


Kategori : Perlengkapan Rumah (Utama) page : 0
Kategori : Perlengkapan Rumah (Utama) page : 1
Kategori : Perlengkapan Rumah (Utama) page : 2
Kategori : Perlengkapan Rumah (Utama) page : 3
Kategori : Perlengkapan Rumah (Utama) page : 4
Kategori : Perlengkapan Rumah (Utama) page : 5
Kategori : Perlengkapan Rumah (Utama) page : 6
Kategori : Perlengkapan Rumah (Utama) page : 7
Kategori : Perlengkapan Rumah (Utama) page : 8


In [ ]:

df.loc[:, 'hargaproduct1'] = df['hargaproduct1'].str.replace('.','', regex=True)
df['hargaproduct2'] = df['hargaproduct2'].apply(lambda x: x if x == 0 else x.replace('.', ''))
df['terjualperbulan'] = df['terjualperbulan'].str.replace('Terjual/ Bulan','', regex=True)
df['terjualperbulan'] = df['terjualperbulan'].str.replace('1RB\+','1000', regex=True)
try:

    df['hargaproduct1'] = df['hargaproduct1'].astype(int)
except:
    print("failed")
    print(df['hargaproduct1'])
df['hargaproduct2'] = df['hargaproduct2'].astype(int)
df['terjualperbulan'] = df['terjualperbulan'].astype(int)
df['average_harga'] = df.apply(lambda x: (x['hargaproduct1'] + x['hargaproduct2'])/2 if x['hargaproduct2'] != 0 else x['hargaproduct1'], axis=1)
df['average_harga'] = df['average_harga'].round().astype(int)
df['total sales per bulan'] = df.apply(lambda x: x['average_harga'] * x['terjualperbulan'], axis=1)
df['total sales per bulan'] = df['total sales per bulan'].round().astype(int)
df.to_excel("data.xlsx")